In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk import FreqDist

import re
import jieba
import math
import time
from collections import Counter
import random

In [2]:
path = '../data/cmn-eng/'

SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

In [3]:
# 工具函数
def deal_en_sen(raw):
    raw.strip()
    letters_only = re.sub("[^a-zA-Z]", " ", raw)
    words = letters_only.lower().split()

    return (" ".join(words))

def deal_zh_sen(raw):
    raw.strip()
    letters_only = re.sub("[^\u4e00-\u9fa5]", "", raw)                        
    
    return(letters_only) 

def word2index(vocab):
    return {word: i for i, word in enumerate(vocab)}

def sen2index(sen, lang):
    global word2index_en
    global word2index_zh
    if lang == 'en':
        return [word2index_en[word] for word in sen.split(' ')]
    else:
        return [word2index_zh[word] for word in list(jieba.cut(sen))]

In [4]:
# 数据预处理
with open(path + 'cmn.txt') as f:
    lines = f.readlines()

In [5]:
en_sens = [deal_en_sen(line.split('\t')[0]) for line in lines]
zh_sens = [deal_zh_sen(line.split('\t')[1]) for line in lines]
pairs = [[en, zh] for en, zh in zip (en_sens, zh_sens)]

In [6]:
pairs[:5]

[['hi', '嗨'], ['hi', '你好'], ['run', '你用跑的'], ['wait', '等等'], ['hello', '你好']]

In [7]:
en_max_len = max([len(x) for x in en_sens])
print(en_max_len)
zh_max_len = max([len(x) for x in zh_sens])
print(zh_max_len)

161
42


In [8]:
# 借助 NLTK 函数
en_word_counts = FreqDist(' '.join(en_sens).split(' '))
# zh_word_counts = FreqDist(list(jieba.cut(''.join(zh_sens))))
en_vocab = set(en_word_counts)
# zh_vocab = set(zh_word_counts)
zh_counts = Counter()
for sen in zh_sens:
    for word in list(jieba.cut(sen)):
        zh_counts[word] += 1
        
zh_vocab = set(zh_counts)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.786 seconds.
Prefix dict has been built succesfully.


In [9]:
# 作用是什么???
# word2index_en = {0: "SOS", 1: "EOS"}
# word2index_zh = {0: "SOS", 1: "EOS"}
# 为了便于训练长度过长的句子都被过滤掉了
MAX_LENGTH = 7
print(len(pairs))
filter_pairs = [pair for pair in pairs if len(pair[0].split(' ')) < MAX_LENGTH and len(list(jieba.cut(pair[1]))) < MAX_LENGTH]
print(len(filter_pairs))
filter_pairs[:6]

19056
9093


[['hi', '嗨'],
 ['hi', '你好'],
 ['run', '你用跑的'],
 ['wait', '等等'],
 ['hello', '你好'],
 ['i try', '让我来']]

In [10]:
# word2index
# { 't' + str(x) : x  for x in range(10)}
word2index_en = word2index(en_vocab)
word2index_zh = word2index(zh_vocab)

In [11]:
sen_vector = [[sen2index(pair[0], 'en'), sen2index(pair[1], 'zh')] for pair in filter_pairs]
sen_vector[:6]

[[[3064], [3716]],
 [[3064], [6496]],
 [[3625], [9462, 12034, 7352, 4108]],
 [[1022], [11788]],
 [[2693], [6496]],
 [[2636, 2138], [8845, 7245, 144]]]

In [12]:
# 模型实现
# seq2seq with attention
# np.array([sen_vector[2][1]]).shape
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, iscuda=False):
        super(EncoderRNN, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.iscuda = iscuda

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers)

    def forward(self, word_inputs, hidden):
        seq_len = len(word_inputs)
        embedded = self.embedding(word_inputs).view(seq_len, 1, -1)
        output, hidden = self.gru()
        return output, hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        if self.iscuda: hidden = hidden.cuda()
        return hidden

In [13]:
class Attn(nn.Module):
    pass

In [14]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, hidden_size, output_size, n_layers=1):
        super(AttnDecoderRNN, self).__init__()
        
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        # self.gru = nn.GRU()
        # self.out = nn.Linear()
        
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)
        
    def forward(self, word_input, last_context, last_hiddden, encoder_outputs):
        word_embedded = self.embedding(word_input).view(1, 1, -1)
        
        rnn_input = torch.cat((), 2)
        rnn_output, hidden = self.gr
        
        attn_weights = 
        context = 
        
        rnn_output = 
        context = 
        output = 
        
        return output, context, hidden, attn_weights

SyntaxError: invalid syntax (<ipython-input-14-80d194a492ad>, line 23)

In [21]:
a = random.choice(sen_vector)

In [30]:
torch.LongTensor([a[0]]).size()

torch.Size([1, 5])

In [15]:
class Attn(nn.Module):
    pass

In [16]:
class AttnDecoderRNN(nn.Module):
    pass

In [17]:
# teacher forcing and clip
teacher_forcing_ratio = 0.5
clip = 5.0

def train():
    pass

In [ ]:
# 训练


In [19]:
str(Variable(torch.LongTensor([[1, 2, 3]])).size())

'torch.Size([1, 3])'

In [20]:
embedding = nn.Embedding(10, 3)

In [21]:
input = Variable(torch.LongTensor([[1,2,3]]))
embedding(input)

Variable containing:
(0 ,.,.) = 
  0.7268 -1.4654  0.4456
  0.3283 -0.0743 -1.6886
 -0.9762 -0.7372  0.4493
[torch.FloatTensor of size 1x3x3]

In [22]:
input = Variable(torch.LongTensor([1,2,3]))
embedding(input)

Variable containing:
 0.7268 -1.4654  0.4456
 0.3283 -0.0743 -1.6886
-0.9762 -0.7372  0.4493
[torch.FloatTensor of size 3x3]

In [23]:
Variable(torch.rand(3))

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.2564  0.3946  0.0448  0.7466  0.3270  0.9833  0.2773  0.5504  0.8895

Columns 9 to 9 
   0.5512

(1 ,.,.) = 

Columns 0 to 8 
   0.3280  0.2546  0.4371  0.9851  0.3637  0.2349  0.2125  0.2808  0.3715

Columns 9 to 9 
   0.6756

(2 ,.,.) = 

Columns 0 to 8 
   0.7372  0.8875  0.3707  0.2788  0.3977  0.8162  0.4258  0.0757  0.4776

Columns 9 to 9 
   0.9132
[torch.FloatTensor of size 3x1x10]

In [35]:
a = Variable(torch.Tensor([[[1, 2, 3, 4, 5, 6, 9999]]]))

In [40]:
a.data[0][0]


    1
    2
    3
    4
    5
    6
 9999
[torch.FloatTensor of size 7]

In [33]:
a.size()

torch.Size([1, 1, 7])

In [34]:
a.data

RuntimeError: cannot call .data on a torch.Tensor: did you intend to use autograd.Variable?

In [55]:
a =torch.LongTensor([[1]])

In [56]:
a


 1
[torch.LongTensor of size 1x1]

In [59]:
torch.LongTensor(a - 1)

TypeError: 'torch.LongTensor' object does not support indexing

In [60]:
a = Variable(torch.Tensor([[[0, 0, 0, 0, 0, 0]]]))

In [61]:
a.topk(1)

(Variable containing:
 (0 ,.,.) = 
   0
 [torch.FloatTensor of size 1x1x1], Variable containing:
 (0 ,.,.) = 
   5
 [torch.LongTensor of size 1x1x1])

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk import FreqDist

import re
import jieba
import math
import time
from collections import Counter
import random

path = '../data/cmn-eng/'

SOS_token = 0
EOS_token = 1
MAX_LENGTH = 10

# 工具函数
def deal_en_sen(raw):
    raw.strip()
    letters_only = re.sub("[^a-zA-Z]", " ", raw)
    words = letters_only.lower().split()

    return (" ".join(words))

def deal_zh_sen(raw):
    raw.strip()
    letters_only = re.sub("[^\u4e00-\u9fa5]", "", raw)                        
    
    return(letters_only) 

def word2index(vocab):
    return {word: i + 1 for i, word in enumerate(vocab)}

def sen2index(sen, lang):
    global word2index_en
    global word2index_zh
    if lang == 'en':
        no_eos = [word2index_en[word] for word in sen.split(' ')]
    else:
        no_eos = [word2index_zh[word] for word in list(jieba.cut(sen))]
    no_eos.append(0)
    return no_eos
    
def as_minutes(s):
    pass

def time_since(since, percent):
    pass
    
# 数据预处理
with open(path + 'cmn.txt') as f:
    lines = f.readlines()
    
en_sens = [deal_en_sen(line.split('\t')[0]) for line in lines]
zh_sens = [deal_zh_sen(line.split('\t')[1]) for line in lines]
pairs = [[en, zh] for en, zh in zip (en_sens, zh_sens)] 

en_max_len = max([len(x) for x in en_sens])
zh_max_len = max([len(x) for x in zh_sens])

# 借助 NLTK 函数
en_word_counts = FreqDist(' '.join(en_sens).split(' '))
# zh_word_counts = FreqDist(list(jieba.cut(''.join(zh_sens))))
en_vocab = set(en_word_counts)
# zh_vocab = set(zh_word_counts)
zh_counts = Counter()
for sen in zh_sens:
    for word in list(jieba.cut(sen)):
        zh_counts[word] += 1
        
zh_vocab = set(zh_counts)

MAX_LENGTH = 7
filter_pairs = [pair for pair in pairs if len(pair[0].split(' ')) < MAX_LENGTH and len(list(jieba.cut(pair[1]))) < MAX_LENGTH]

word2index_en = word2index(en_vocab)
word2index_en[0] = 'EOS'
word2index_zh = word2index(zh_vocab)
word2index_zh[0] = 'EOS'
sen_vector = [[sen2index(pair[0], 'en'), sen2index(pair[1], 'zh')] for pair in filter_pairs]

In [15]:
sen_vector[:6]

[[[5420, 0], [2934, 0]],
 [[5420, 0], [630, 0]],
 [[792, 0], [2905, 3815, 9543, 11548, 0]],
 [[970, 0], [9899, 0]],
 [[5632, 0], [630, 0]],
 [[3566, 354, 0], [12825, 5960, 3425, 0]]]

In [8]:
a= [1, 3]
print(a.append(0))

None


In [17]:
a = torch.LongTensor([0])

In [18]:
a


 0
[torch.LongTensor of size 1]

In [19]:
a[0]

0